### Demonstration of Quantum Key Distribution with the Ekert 91 Protocol

Algorithm -
1. First generate the a maximally entangled qubit pair |psi+> = 1/root(2) * (|01> + |10>)
2. Send one qubit to Alice and one qubit to Bob
3. Both Alice and Bob perform their measurement and make the measurement bases public.
4. According to the new information obtained, a sifted key is created, which can be used for secure communication

Modification to this Algorithm (because we get only 1 quantum computer) -
1. First generate the a maximally entangled qubit pair |psi+> = 1/root(2) * (|01> + |10>)
2. Take the measurement bases from Alice and Bob
3. Perform measurement, and send the measurement results to Alice and Bob respectively
4. Note that Alice and Bob do not have each other's measurement outcomes, they have only theirs
5. The measurement bases are made public, and the sifted key is obtained

In [24]:
import os
import random
from apikey import token

from qiskit import execute
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit

from quantuminspire.credentials import enable_account, get_authentication
from quantuminspire.qiskit import QI

print("Process Complete!")

Process Complete!


In [2]:
# Initiating the Quantum Inspire Account -
enable_account(token)
authentication = get_authentication()
QI.set_authentication()
qi_backend = QI.get_backend('QX single-node simulator')

print("Process Complete!")

Process Complete!


In [3]:
# Step 1 - Creating EPR Pair
q = QuantumRegister(2)
b = ClassicalRegister(2)
circuit = QuantumCircuit(q, b)

circuit.h(q[0])
circuit.x(q[1])
circuit.cx(q[0], q[1])

print("Process Complete!")

Process Complete!


#### Measurement

Ai = Alice's measurement  
Bi = Bob's measurement  

A more robust approach would involve the following bases -   
A1 = Alice measures along the Z basis  
A2 = Alice measures along the X basis  
A3 = Alice measures along 1/root(2) * (Z + X)  

B1 = Bob measures along the Z basis  
B2 = Bob measures along the 1/root(2) * (Z - X)  
B3 = Bob measures along the 1/root(2) * (Z + X)  

Going with the following bases options for simplicity -  
A1 = Alice measures along the Z basis  
A2 = Alice measures along the X basis  
B1 = Bob measures along the Z basis  
B2 = Bob measures along the X basis  

In [4]:
# Actual protocol's measurements are done as follows -
# Giving q0 to Alice and q1 to Bob
# can be 1,2 or 3
alice = 1 
bob = 1

if alice == 2:
    circuit.x(q[0])

if alice == 3:
    circuit.s(q[0])
    circuit.h(q[0])
    circuit.t(q[0])
    circuit.h(q[0])

if bob == 2:
    circuit.s(q[1])
    circuit.h(q[1])
    circuit.t(q[1]).inverse()
    
if bob == 3:
    circuit.s(q[1])
    circuit.h(q[1])
    circuit.t(q[1])
    circuit.h(q[1])
    
circuit.measure(q, b)
qi_job = execute(circuit, backend=qi_backend, shots=1)
qi_result = qi_job.result()
result = qi_result.get_counts(circuit)

print(result)

{'01': 137, '10': 119}


In [32]:
def entangle_qubits(qc, q, q1, q2):
    """
    Puts the specified qubits in a maximally entangled state - |psi+>
    |psi+> = 1/root(2) * (|01> + |10>)
    
    Params:
    qc = quantum circuit object
    q  = qubits
    q1, q2 = the two qubits to be entangled
    """
    
    qc.h(q[q1])
    qc.x(q[q2])
    qc.cx(q[q1], q[q2])

def perform_measurement(qc, q, b, basis1, basis2):
    """
    Returns one bit of the key generated using the Ekert 91 protocol
    basis1 = Measurement basis of Alice
    basis2 = Measurement basis of Bob
    """
    
    alice = basis1
    bob = basis2

    if alice == 2:
        qc.x(q[0])

    if bob == 2:
        qc.x(q[1])

    qc.measure(q, b)
    qi_job = execute(qc, backend=qi_backend, shots=1)
    qi_result = qi_job.result()
    result = qi_result.get_counts(qc)

    outcome = list(result.keys())[0]
    return outcome[0], outcome[1]
    
def ekert91(basis1, basis2):
    
    q = QuantumRegister(2)
    b = ClassicalRegister(2)
    qc = QuantumCircuit(q, b)
    
    # Step 1 - Creating EPR Pair
    entangle_qubits(qc, q, 0, 1)
    
    # Step 2 - 
    outcome = perform_measurement(qc, q, b, basis1, basis2)
    return outcome[0], outcome[1]

def generateKey(bases1, bases2):
    if len(bases1) != len(bases2) or len(bases1) != 10:
        return False
    
    aliceOutcome = []
    bobOutcome = []
    
    # Taking 10 bits for our unsifted key
    n = 10
    for i in range(n):
        result = ekert91(bases1[i], bases2[i])
        aliceOutcome.append(int(result[0]))
        bobOutcome.append(int(result[1]))
    
    # Send the outcomes to the respective receivers
    # After this, Alice and Bob will publicly announce their measurement 
    # bases and obtain the sifted key!
    return aliceOutcome, bobOutcome

def generateBases(n):
    """
    Function to generate a psuedorandom sequence of bases 
    """
    bases = []
    
    for i in range(n):
        bases.append(random.randint(1,2))

    return bases

print("Process Complete!")

Process Complete!


In [36]:
"""
If both Alice and Bob measure along the X basis, then if Alice's bit is 0 then Bob's bit is also 0.
Similarly, if Alice's bit is 1, then Bob's bit is also 1.

If both Alice and Bob measure along the Z basis, then their outcomes are anti-correlated, i.e.
If Alice = 1, then Bob = 0
If Alice = 0, then Bob = 1

In this case, both have decided that if they measure along the Z basis, then Bob will flip his bit.
"""

def getSiftedKey(key, user, bases1, bases2):
    siftedKey = []
    
        
    for i in range(len(bases1)):
        if bases1[i] == bases2[i]:
            if bases1[i] == 1: # Both measured along the Z basis, flip Bob's bit
                if user == "Bob":
                    if key[i] == 0:
                        siftedKey.append(1)
                    else:
                        siftedKey.append(0)
                else:
                    siftedKey.append(key[i])
            else: # Both measured along the X basis
                siftedKey.append(key[i])
    
    return siftedKey


In [41]:
# Execution of the whole protocol -

b1 = generateBases(10)
b2 = generateBases(10)

print("Alice Bases: ", b1)
print("Bob Bases  : ", b2)

AliceKey, BobKey = generateKey(b1, b2)
print("Alice's Key: ", AliceKey)
print("Bob's Key  : ", BobKey)

# Now Alice and Bob have their own measurements. They have also publicly announced their bases
# They need to individually find the sifted keys from what they receive
Alice_sifted_key = getSiftedKey(AliceKey, "Alice", b1, b2)
Bob_sifted_key = getSiftedKey(BobKey, "Bob", b1, b2)

print("Alice's Sifted Key: ", Alice_sifted_key)
print("Bob's Sifted Key  : ", Bob_sifted_key)

Alice Bases:  [1, 1, 1, 2, 2, 2, 1, 1, 1, 1]
Bob Bases  :  [1, 1, 1, 2, 1, 1, 1, 1, 2, 1]
Alice's Key:  [0, 0, 1, 0, 0, 1, 1, 0, 1, 0]
Bob's Key  :  [1, 1, 0, 1, 0, 1, 0, 1, 1, 1]
Alice's Sifted Key:  [0, 0, 1, 0, 1, 0, 0]
Bob's Sifted Key  :  [0, 0, 1, 1, 1, 0, 0]


As expected, both their sifted keys are the same. (barring some noise)  
Thus, Quantum Key Distribution has been demonstrated